In [2]:
import pandas as pd

## Load & Pre-Process Dataset

In [28]:
data_df = pd.read_csv('../data/raw/amazon_popular_books_dataset.csv', usecols=['title', 'categories', 'rating', 'reviews_count'])

# Pre-process Dataset
data_df['rating'] = pd.to_numeric(data_df['rating'].str.slice(0, 3))

# Add weighted rating
m = 100  # set the minimum number of reviews to 100
C = data_df['rating'].mean()  # calculate the mean rating across all books

# define a function to calculate the weighted rating
def weighted_rating(row):
    v = row['reviews_count']
    R = row['rating']
    return (v / (v+m)) * R + (m / (v+m)) * C

# apply the function to each row of the dataframe to calculate the weighted rating
data_df['weighted_rating'] = data_df.apply(weighted_rating, axis=1)

data_df

,rating,reviews_count,title,categories,weighted_rating
0,4.6,13451,Wuthering Heights (Collins Classics),"[""Books"",""Literature & Fiction"",""Genre Fiction""]",4.600165
1,4.8,16628,THE DAYS THE CRAYONS QUIT,"[""Books"",""Children's Books"",""Literature & Fict...",4.798938
2,4.8,11275,War Lord: Book 13 (The Last Kingdom Series),"[""Books"",""Literature & Fiction"",""Genre Fiction""]",4.798438
3,4.8,15520,Code Name Bananas: The hilarious and epic new ...,"[""Books"",""Children's Books"",""Literature & Fict...",4.798863
4,4.8,10884,Skincare: The award-winning ultimate no-nonsen...,"[""Books"",""Crafts, Hobbies & Home"",""Home Improv...",4.798383
...,...,...,...,...,...
2264,4.2,17923,Unspeakable Things,"[""Books"",""Mystery, Thriller & Suspense"",""Thril...",4.202343
2265,4.8,25304,What to Expect When You’re Expecting,"[""Books"",""Health, Fitness & Dieting"",""Women's ...",4.799301
2266,4.7,11040,The Home Edit: A Guide to Organizing and Reali...,"[""Books"",""Crafts, Hobbies & Home"",""Home Improv...",4.699303
2267,4.4,28030,The Family Upstairs: A Novel,"[""Books"",""Mystery, Thriller & Suspense"",""Thril...",4.400790
